# Overview

> The goal of this notebook is to show how we can create a list of medications for a single patient.

As the first step in converting FHIR resources to the CDISC "Concomitant/Prior Medications" CM domain, we'll create a `Bundle` containing one `Patient` and any number of `MedicationAdministration`, `MedicationDispense`, `MedicationRequest` and `MedicationStatement` resources.

So that subsequent use of this bundle doesn't have to make any FHIR server requests, the bundle will also contain;
- `Medication` referenced by `medicationReference`
- `Condition` and/or `Observation` referenced by `reasonReference`

## TODO: Remove non-concomitant medications from the list

Identifying concomitant medications might get quite complicated - I'm assuming we won't be able to cover all logic needed when pulling data from the FHIR servers. I think it makes sense to pull all medications, then add a concomitant medication filter as a subsequent step.

### How are we defining concomitant medications? 

Any medication  
- that is not the medication being investigated
- that is being taken while a patient is participating in a study

We might also want to list subset of concomitant medications - i.e. thoes listed in exclusion criteria, relevant medications that the study would like to follow (e.g. concomitant use of ACE inhibitors might be important but single dose paracetamil might not).

To know if the medication was being taken while the patient was/is participating in a study, we could

could compare study participation
- study participation from `ResearchSubject.period`
- study duration from `ResearchStudy.period`
    - if either start or end date are missing from `ResearchSubject.period`
- user specified start and end date
    - if `ResearchStudy` etc are not in FHIR?
    
with start and end time of medication "administration"
- `MedicationStatement.effectiveX`, `MedicationStatement.dateAsserted`, `MedicationStatement.dosage`
    - don't forget `MedicationX.status` not-taken etc
- `MedicationRequest.authoredOn`, `MedicationRequest.encounter`, `MedicationRequest.dosageInstruction`, `MedicationRequest.basedOn`, `MedicationRequest.dispenseRequest` ...
    - Don't forget `MedicationRequest.doNotPerform`
- `MedicationDispense.daysSupply`, `MedicationDispense.whenPrepared`, `MedicationDispense.whenHandedOver`, `MedicationDispense.dosageInstruction`, `MedicationDispense.partOf`, `MedicationDispense.authorizingPrescription` 
- `MedicationAdministration.effectiveX`, partOf, supportingInformation ...


## TODO: think about a "human in the loop" to help with things &uarr; that will be hard to reliably automate

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pete88b/smart-on-fhir-client-py-demo/blob/main/vulcan_per_patient_medication_bundle.ipynb)

This notebook follows on from [vulcan_medication_bundle_getting_started.ipynb](https://colab.research.google.com/github/pete88b/smart-on-fhir-client-py-demo/blob/main/vulcan_medication_bundle_getting_started.ipynb) - which explains
- Why we are not using `List` and
- Why we are reading FHIR resources as raw JSON

## Next steps

Might we want to
- define some kind of order of entries in the bundle
- think about how we handle resources that fail validation 
    - We can use https://inferno.healthit.gov/validator/ to validate the bundes created
        - TODO: Can we discuss how we want to action this output?

## Required resources?

The CM tab (https://wiki.cdisc.org/display/FHIR2CDISCUG/FHIR+to+CDISC+Mapping+User+Guide+Home FHIR-to-CDISC Mappings xlsx) lists the following resources;

- ~~`ResearchSubject` with `ResearchStudy`~~ NOT YET?
- `Subject`
- `MedicationStatement`
- `MedicationRequest`
- `MedicationDispense`
- `MedicationAdministration`
- ~~`Immunization`~~ don't think we're doing immunization yet?
- `Medication` referenced by `medicationReference`
- `Condition` or `Observation` referenced by `reasonReference`

TODO: For now, I'm just pulling the resources that Jay highlighted as required - we can easily add the others (o:

In [ ]:
import IPython, json, requests
from datetime import datetime, timezone
from collections import Counter
from pathlib import Path

In [ ]:
api_base = 'http://hapi.fhir.org/baseR4'

In [ ]:
def get_bundle_as_raw_json(api_base, resource_type, url_suffix=None):
    "GET a bundle of resources of a specific type"
    url=f'{api_base}/{resource_type}'
    if url_suffix is not None:
        url+=url_suffix
    print('GET',url)
    return requests.get(url).json()

def get_next_bundle_as_raw_json(json_response):
    "GET the next set of results"
    if len(json_response['link']) == 0: return None
    url = json_response['link'][1]['url']
    print('GET',url)
    return requests.get(url).json()

The following cell shows how `get_bundle_as_raw_json` and `get_next_bundle_as_raw_json` can be used to read paged search results.

In [ ]:
json_response = get_bundle_as_raw_json(api_base, 'ResearchStudy')
page_count = 1
while json_response is not None:    
    for entry in json_response['entry']:
        resource = entry.get('resource', {})
        print('ResearchStudy:id', resource.get('id', 'missing'), resource.get('title', '')[:60])
    if page_count > 2: break # pull 3 pages at most to make testing fast
    json_response = get_next_bundle_as_raw_json(json_response)
    page_count += 1

GET http://hapi.fhir.org/baseR4/ResearchStudy
ResearchStudy:id 2492775 Double blind, placebo-controlled trial of a new class of art
ResearchStudy:id 1164317 Adjuvant Aspirin Treatment in PIK3CA Mutated Colon Cancer Pa
ResearchStudy:id 1164321 A Phase III, Multicenter, Randomized, Open-Label Study Compa
ResearchStudy:id 2140627 
ResearchStudy:id 2126241 Projectathon 2021 Machbarkeitsanfrage
ResearchStudy:id 2126240 Projectathon 2021 Machbarkeitsanfrage
ResearchStudy:id 2114074 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2114070 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111327 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111323 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111319 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111315 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111311 Saf

We need to find a research study that has subjects with medication ...

It would be nice if we could use something like

```
get_bundle_as_raw_json(api_base, 'ResearchStudy', f'?_has:ResearchSubject:study:identifier=*')
```

but ... we can't used a wildcard for the identifier and we have to include a ResearchSubject field in the search.

So ... we could try studies from the list above, or list research subjects and work backwards.

In the following cell, we list all research subjects and should see `ResearchSubject:id` 1171863 which has a reference to `ResearchStudy:id` 1171831

In [ ]:
get_bundle_as_raw_json(api_base, 'ResearchSubject')

GET http://hapi.fhir.org/baseR4/ResearchSubject


{'resourceType': 'Bundle',
 'id': '9064c2f6-b0a4-4e2b-a5d5-431b64e961db',
 'meta': {'lastUpdated': '2021-08-26T15:56:29.765+00:00'},
 'type': 'searchset',
 'total': 258,
 'link': [{'relation': 'self',
   'url': 'http://hapi.fhir.org/baseR4/ResearchSubject'},
  {'relation': 'next',
   'url': 'http://hapi.fhir.org/baseR4?_getpages=9064c2f6-b0a4-4e2b-a5d5-431b64e961db&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset'}],
 'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/ResearchSubject/1171863',
   'resource': {'resourceType': 'ResearchSubject',
    'id': '1171863',
    'meta': {'versionId': '1',
     'lastUpdated': '2020-05-23T11:26:17.900+00:00',
     'source': '#HiDFuYuBY6XnuyTd'},
    'status': 'candidate',
    'period': {'start': '2020-01-14T18:52:35+01:00'},
    'study': {'reference': 'ResearchStudy/1171831'},
    'individual': {'reference': 'Patient/0c4a1143-8d1c-42ed-b509-eac97d77c9b2'}},
   'search': {'mode': 'match'}},
  {'fullUrl': 'http://hapi.fhir.org/baseR4/R

Feel free to try other studies but this one looks pretty good - thanks Jozef (o:

In [ ]:
research_study_id = 1171831
# Note: &_revinclude=* gives us everything refering to the study
get_bundle_as_raw_json(api_base, 'ResearchStudy', f'?_id={research_study_id}&_revinclude=*')

GET http://hapi.fhir.org/baseR4/ResearchStudy?_id=1171831&_revinclude=*


{'resourceType': 'Bundle',
 'id': 'aa9c53fc-9cd7-4688-998d-2310252848c4',
 'meta': {'lastUpdated': '2021-08-26T15:56:30.117+00:00'},
 'type': 'searchset',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://hapi.fhir.org/baseR4/ResearchStudy?_id=1171831&_revinclude=*'}],
 'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/ResearchStudy/1171831',
   'resource': {'resourceType': 'ResearchStudy',
    'id': '1171831',
    'meta': {'versionId': '1',
     'lastUpdated': '2020-05-23T06:17:19.321+00:00',
     'source': 'http://www.xml4pharma.com/SyntheaStudy#RXlgic5zF0w2KKQT'},
    'language': 'en',
    'identifier': [{'use': 'official',
      'system': 'http://www.xml4pharma.com/Synthea',
      'value': 'SyntheaStudy'}],
    'title': 'Simple Covid-19 observational study',
    'status': 'active',
    'condition': [{'coding': [{'system': 'http://snomed.info/sct',
        'code': '840539006',
        'display': 'Covid-19'}]}],
    'contact': [{'name': 'Jozef Aerts',
      'telecom': [{

We can pick a patient from the above bundle and pull medication requests &darr;

In [ ]:
# 'subject': {'reference': 'Patient/0c4a1143-8d1c-42ed-b509-eac97d77c9b2'
get_bundle_as_raw_json(api_base, 'MedicationRequest', '?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2')

GET http://hapi.fhir.org/baseR4/MedicationRequest?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2


{'resourceType': 'Bundle',
 'id': '6833e69b-ecdb-4c2b-b904-4a69405fad9c',
 'meta': {'lastUpdated': '2021-08-26T15:56:30.514+00:00'},
 'type': 'searchset',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://hapi.fhir.org/baseR4/MedicationRequest?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2'}],
 'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/MedicationRequest/08250384-7d1d-4553-b9c7-75a78390c184',
   'resource': {'resourceType': 'MedicationRequest',
    'id': '08250384-7d1d-4553-b9c7-75a78390c184',
    'meta': {'versionId': '1',
     'lastUpdated': '2020-03-24T18:00:19.621+00:00',
     'source': '#wjBsr5FxmhpDPleG',
     'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-medicationrequest'],
     'tag': [{'system': 'https://smarthealthit.org/tags',
       'code': 'Covid19 synthetic population from Synthea'}]},
    'status': 'active',
    'intent': 'order',
    'medicationCodeableConcept': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm

In [ ]:
def extract_references(single_resource_bundle):
    "Return a list of relative references e.g. `['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8']`"
    result = []
    for entry in single_resource_bundle['entry']:
        resource = entry.get('resource', {})
        for f in ['medicationReference', 'reasonReference']:
            if f in resource:
                references = resource[f]
                if not isinstance(references, list): references = [references]
                for r in references:
                    if 'reference' in r: 
                        # TODO: check that we have a relative reference or handle other kinds too
                        result.append(r['reference'])
    return result # TODO: make this a set of unique references

In [ ]:
def read_by_reference(api_base, reference):
    "Return a resource read from a FHIR server by reference, as a list containg a single bundle entry"
    resource_type, id = reference.split('/')
    single_resource_bundle = get_bundle_as_raw_json(api_base, resource_type, f'?_id={id}')
    total = single_resource_bundle['total']
    if total != 1:
        raise Exception(f'Expected a single resource for {reference} but found {total}')
        # TODO: we might want to print a warning and return an empyt list if this is too strict
    return single_resource_bundle['entry']

If a resource has a reference like
```
'medicationReference': {'reference': 'Medication/bac1387e-3655-4e03-982f-7210faa21ea8'},
```
we can read the referenced resource (a `Medication` in this case) with http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8

In [ ]:
test_id = 'bac1387e-3655-4e03-982f-7210faa21ea8'
assert read_by_reference(api_base, f'Medication/{test_id}')[0]['resource']['id'] == test_id

GET http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8


In [ ]:
def create_single_patient_medication_bundle(api_base, patient_id):
    "Return a Bundle containing one Patient and any number of MedicationX resources"
    result = dict(resourceType='Bundle', 
              type='collection', 
              timestamp=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'), 
              entry=[])
    references = []
    for resource_type, url_suffix in [
            ['Patient', f'?_id={patient_id}'],
            ['MedicationRequest', f'?subject={patient_id}'],
            ['MedicationDispense', f'?subject={patient_id}'],
            ['MedicationAdministration', f'?subject={patient_id}'],
            ['MedicationStatement', f'?subject={patient_id}']]:
        single_resource_bundle = get_bundle_as_raw_json(api_base, resource_type, url_suffix)
        if single_resource_bundle['total'] > 0:
            result['entry'].extend(single_resource_bundle['entry'])
            references.extend(extract_references(single_resource_bundle))
    for reference in set(references):
        result['entry'].extend(read_by_reference(api_base, reference))
    return result

In [ ]:
create_single_patient_medication_bundle(api_base, '0c4a1143-8d1c-42ed-b509-eac97d77c9b2')

GET http://hapi.fhir.org/baseR4/Patient?_id=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationRequest?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationDispense?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationAdministration?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationStatement?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/Condition?_id=1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8
GET http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8


{'resourceType': 'Bundle',
 'type': 'collection',
 'timestamp': '2021-08-26T15:56:30Z',
 'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/Patient/0c4a1143-8d1c-42ed-b509-eac97d77c9b2',
   'resource': {'resourceType': 'Patient',
    'id': '0c4a1143-8d1c-42ed-b509-eac97d77c9b2',
    'meta': {'versionId': '2',
     'lastUpdated': '2020-04-16T18:52:26.418+00:00',
     'source': '#eBILm7Zu9t2rzsa7',
     'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient'],
     'tag': [{'system': 'https://smarthealthit.org/tags',
       'code': 'Covid19 synthetic population from Synthea'}]},
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by \n         \n            <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: 2177cffc\n .   Person seed: 4125951794022957444  Population seed: 12345\n        </div>'},
    'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',

In [ ]:
def save_single_patient_medication_bundle(bundle):
    "Write a patient medication bundle to file."
    Path('data').mkdir(exist_ok=True)
    patient = bundle['entry'][0]['resource']
    if patient['resourceType'] != 'Patient':
        raise Exception(f'expected a patient but found {patient}')
    patient_id = patient['id']
    f_name = f'data/patient_medication_bundle_{patient_id}.json'
    with open(f_name, 'w') as f:
        json.dump(bundle, f, indent=2)
    print('Bundle saved to', f_name)

Now we can save the JSON bundle to file to pass on to the next step of the process (o:

Note: you can download files via the file explorer in colab

In [ ]:
save_single_patient_medication_bundle(
    create_single_patient_medication_bundle(api_base, '0c4a1143-8d1c-42ed-b509-eac97d77c9b2'))

GET http://hapi.fhir.org/baseR4/Patient?_id=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationRequest?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationDispense?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationAdministration?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationStatement?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/Condition?_id=1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8
GET http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8
Bundle saved to data/patient_medication_bundle_0c4a1143-8d1c-42ed-b509-eac97d77c9b2.json


The bundle we are creating is a `collection`, so we need to remove `search` elements from each `entry`. This removes some validation errors reported by https://inferno.healthit.gov/validator/ - thanks Mike (o:

TODO: Do we care what the `search` element is telling us? i.e. what if it's not `match`?

In [ ]:
def handle_entry_search(bundle):
    "Remove `search` elements from each `entry`"
    for entry in bundle['entry']:
        if 'search' in entry: del entry['search']
    return bundle

# Medication status filter

We want to remove medications if the status tells us the medication was not or will not be taken. 

- https://www.hl7.org/fhir/valueset-medicationrequest-status.html
- https://www.hl7.org/fhir/valueset-medicationdispense-status.html
- https://www.hl7.org/fhir/valueset-medication-admin-status.html
- https://www.hl7.org/fhir/valueset-medication-statement-status.html


## List statuses that we want to remove from the bundle

- MedicationRequest (Include: active, on-hold, completed, entered-in-error, unknown)
    - cancelled 
        - The prescription has been withdrawn before any administrations have occurred
    - stopped 
        - Actions implied by the prescription are to be permanently halted, before all of the administrations occurred. 
        - This is a ? **halted, before all ...** i.e. might some of the administrations occured
    - draft
        - The prescription is not yet 'actionable'
     
     
- MedicationDispense (Include: on-hold, completed, unknown)
    - preparation	
        - The core event has not started yet, 
    - in-progress
        - The dispensed product is ready for pickup
    - cancelled	
        - The dispensed product was not and will never be picked up by the patient
    - entered-in-error	
        - The dispense was entered in error and therefore nullified
    - stopped	
        - Actions implied by the dispense have been permanently halted, before all of them occurred
        - This is a ? **hatled, before all ...** i.e. might some of the actions occured
    - declined	
        - The dispense was declined and not performed.
        
- MedicationAdministration (Include: in-progress, on-hold, completed, stopped, unknown)
    - not-done	
        - The administration was terminated prior to any impact on the subject
    - entered-in-error	
        - The administration was entered in error and therefore nullified
        
- MedicationStatement (Include: active, completed, entered-in-error, intended, stopped, on-hold, unknown)
    - not-taken
        - The medication was not consumed by the patient

In [ ]:
def filter_bundle(bundle, filter_fn):
    "Apply a filter function to a bundle in-place"
    bundle['entry'] = [e for e in bundle['entry'] if filter_fn(e)]
    return bundle

In [ ]:
def medication_status_filter(entry):
    "Remove medications if the status tells us the medication was not or will not be taken"
    statuses_to_remove_map = dict(
        MedicationRequest=['cancelled','stopped','draft'],
        MedicationDispense=['preparation','in-progress','cancelled','entered-in-error','stopped','declined'],
        MedicationAdministration=['not-done','entered-in-error'],
        MedicationStatement=['not-taken'])
    resource = entry.get('resource', {})
    resourceType, status = resource.get('resourceType'), resource.get('status')
    statuses_to_remove = statuses_to_remove_map.get(resourceType)
    if statuses_to_remove is not None and status in statuses_to_remove:
        print('Removing', resourceType, 'with status', status)
        return False
    return True

In [ ]:
def remove_patient_filter(entry):
    resource = entry['resource']
    if resource['resourceType'] == 'Patient': return False
    return True

We could create medication bundles for all subjects in a study with something like.

Note: We're starting to build a bundle processing pipeline (by adding calls to `handle_entry_search` and `medication_status_filter`) - and we'll add more functions like this to remove non-concomitant medications etc

In [ ]:
study_and_subject_bundle = get_bundle_as_raw_json(api_base, 'ResearchStudy', 
                                                  f'?_id={research_study_id}&_revinclude=ResearchSubject:study')
for i, entry in enumerate(study_and_subject_bundle['entry']):
    resource = entry.get('resource', {})
    if resource.get('resourceType', 'unk') != 'ResearchSubject': continue
    patient_reference = resource.get('individual',{}).get('reference')[8:]
    bundle = create_single_patient_medication_bundle(api_base, patient_reference)
    bundle = handle_entry_search(bundle)
    bundle = filter_bundle(bundle, medication_status_filter)
    save_single_patient_medication_bundle(bundle)
    if i>1: break # stop early (o:

GET http://hapi.fhir.org/baseR4/ResearchStudy?_id=1171831&_revinclude=ResearchSubject:study
GET http://hapi.fhir.org/baseR4/Patient?_id=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationRequest?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationDispense?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationAdministration?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationStatement?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/Condition?_id=1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8
GET http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8
Bundle saved to data/patient_medication_bundle_0c4a1143-8d1c-42ed-b509-eac97d77c9b2.json
GET http://hapi.fhir.org/baseR4/Patient?_id=1b0580b9-1ee3-4353-b555-64c797d57564
GET http://hapi.fhir.org/baseR4/MedicationRequest?subject=1b0580b9-1ee3-4353-b555-64c797d57564
GET h